In [ ]:
!git clone https://github.com/AI4Finance-LLC/FinRL-Library.git

In [ ]:
%cd FinRL-Library/
!pip install -r requirements.txt

In [ ]:
!pip install -U ipython
# !pip install colorama

In [ ]:
%cd FinRL-Library/

/content/FinRL-Library


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.6/dist-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [ ]:
from finrl.config.configuration import Configuration
from finrl.config.directory_operations import create_userdata_dir
from finrl.commands import start_download_cryptodata, start_download_stockdata

In [ ]:
#### CREATE USER DATA DIRECTORY IN DESIGNATED PATH, IF NO NAME INDICATED DEFAULT TO user_data
####### create dir to false if only to check existence of directory
create_userdata_dir("./user_data",create_dir=True)


# ###### Pull Configuration File (using finrl/config/configuration.py)
config = Configuration.from_files(["./notebooks/config.json"])

In [ ]:
config.get("exchange").get("pair_whitelist")

In [ ]:
from finrl.commands import start_download_cryptodata, start_download_stockdata, start_list_markets

#ARGS_LIST_PAIRS = ["exchange", "print_list", "list_pairs_print_json", "print_one_column",
#                   "print_csv", "base_currencies", "quote_currencies", "list_pairs_all"]

# ARGS_LIST_PAIRS = {"exchange":config.get("exchange").get("name"), "quote_currencies":"BNB"}
ARGS_LIST_PAIRS = {"exchange":config.get("exchange").get("name")}

x = start_list_markets(ARGS_LIST_PAIRS)


In [ ]:
from finrl.tools.coin_search import *
import json
#Search top Selling Coins based on Volume
coins = coinSearch("BTC", top=100)
print(coins)
#Add them to config file Pair_whitelist
coins_to_json("./notebooks/config.json", coins)

# reintialize config
config = Configuration.from_files(["./notebooks/config.json"])

#make sure the pairs are equal...
print(config["exchange"]["pair_whitelist"])

['DREP/BTC', 'STMX/BTC', 'TROY/BTC', 'SC/BTC', 'CKB/BTC', 'TRX/BTC', 'PHB/BTC', 'IOST/BTC', 'CELR/BTC', 'XVG/BTC', 'ONE/BTC', 'TCT/BTC', 'IOTX/BTC', 'COS/BTC', 'CHZ/BTC', 'VET/BTC', 'MITH/BTC', 'DOGE/BTC', 'SNGLS/BTC', 'OST/BTC', 'REEF/BTC', 'TNB/BTC', 'RVN/BTC', 'NBS/BTC', 'GTO/BTC', 'ARPA/BTC', 'QKC/BTC', 'CND/BTC', 'WPR/BTC', 'MTH/BTC', 'YOYOW/BTC', 'SNM/BTC', 'CDT/BTC', 'FUN/BTC', 'AKRO/BTC', 'STPT/BTC', 'JST/BTC', 'DGB/BTC', 'DOCK/BTC', 'VITE/BTC', 'ZIL/BTC', 'FOR/BTC', 'MATIC/BTC', 'ANKR/BTC', 'BLZ/BTC', 'DATA/BTC', 'RSR/BTC', 'STEEM/BTC', 'GO/BTC', 'MDT/BTC', 'HBAR/BTC', 'PERL/BTC', 'TFUEL/BTC', 'ONT/BTC', 'AMB/BTC', 'DUSK/BTC', 'ROSE/BTC', 'REQ/BTC', 'CTSI/BTC', 'BTCDOWN/USDT', 'IRIS/BTC', 'BTS/BTC', 'POA/BTC', 'NKN/BTC', 'COTI/BTC', 'CTXC/BTC', 'AERGO/BTC', 'APPC/BTC', 'EOS/BTC', 'XLM/BTC', 'NAS/BTC', 'CHR/BTC', 'AUDIO/BTC', 'RCN/BTC', 'QSP/BTC', 'CRV/BTC', 'XEM/BTC', 'SAND/BTC', 'BAT/BTC', 'SKL/BTC', 'QLC/BTC', 'FIO/BTC', 'BRD/BTC', 'MTL/BTC', 'SNT/BTC', 'DLT/BTC', 'ADA/BTC',

In [ ]:
ARGS_DOWNLOAD_DATA = {'config': ['./notebooks/config.json'], 'datadir': None, 
                      'user_data_dir': None, 'pairs': None, 'pairs_file': None, 
                      'days': 1825, 'timerange': None, 
                      'download_trades': False, 'exchange': 'binance', 
                      'timeframes': ['1d'], 'erase': False, 
                      'dataformat_ohlcv': None, 'dataformat_trades': None}

# ######## downloads data to our local data repository as dictated by our config, or we could overide it using 'datadir'
start_download_cryptodata(ARGS_DOWNLOAD_DATA)

In [ ]:
from finrl.config import setup_utils_configuration
from finrl.state import RunMode

ARGS_DOWNLOAD_DATA = {'config': ['./notebooks/config.json'], 'datadir': None, 
                      'user_data_dir': None, 'pairs': None, 'pairs_file': None, 
                      'days': 1825, 'timerange': None, 
                      'download_trades': False, 'exchange': 'binance', 
                      'timeframes': '1d', 'erase': False, 
                      'dataformat_ohlcv': None, 'dataformat_trades': None}

### Adds to config ARGS stats for further use through config

config = setup_utils_configuration(ARGS_DOWNLOAD_DATA, RunMode.UTIL_EXCHANGE)
print(config.get("timeframes"))

1d


In [ ]:
from finrl.data.fetchdata import FetchData
import pandas as pd
from finrl.config import TimeRange

In [ ]:
df = FetchData(config).fetch_data_crypto()
df

In [ ]:
df.head()

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config["TECHNICAL_INDICATORS_LIST"],
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)
processed.columns

Successfully added technical indicators


Index(['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma'],
      dtype='object')

In [ ]:
##### SAVING processed DF for future use or in case COLAB shuts down

from datetime import date
processed.to_pickle("processed_btc_full_"+str(date.today()))
processed.to_csv("processed_btc_full_csv_"+str(date.today()))

In [ ]:
print(processed.head())
processed.shape

        date          open          high  ...  dx_30  close_30_sma  close_60_sma
0 2020-01-16  4.200000e-07  6.400000e-07  ...  100.0  4.700000e-07  4.700000e-07
1 2020-01-17  4.700000e-07  5.200000e-07  ...  100.0  4.100000e-07  4.100000e-07
2 2020-01-18  3.600000e-07  3.800000e-07  ...  100.0  3.833333e-07  3.833333e-07
3 2020-01-19  3.300000e-07  3.400000e-07  ...  100.0  3.625000e-07  3.625000e-07
4 2020-01-20  2.900000e-07  3.100000e-07  ...  100.0  3.460000e-07  3.460000e-07

[5 rows x 15 columns]


(74850, 15)

In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max(), freq="1d").astype(str))
combination = list(itertools.product(list_date,list_ticker))
list_date

In [ ]:
processed["date"] = processed["date"].astype(str)
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
# processed_full = processed_full.dropna()
processed_full = processed_full.fillna(0)
processed_full = processed_full.reset_index(drop=True)
processed_full.sort_values(['date','tic'],ignore_index=True)
processed_full.date = pd.to_datetime(processed_full.date)
processed_full["day"] = processed_full["date"].dt.dayofweek

In [ ]:
processed_full.head(5)

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,day
0,2017-08-27,DREP_BTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
1,2017-08-27,STMX_BTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
2,2017-08-27,TROY_BTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
3,2017-08-27,SC_BTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,2017-08-27,CKB_BTC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6


In [ ]:
processed_full.tail()

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,day
127095,2021-02-17,ENJ_BTC,0.000011,0.000011,0.000010,0.000011,13893935.0,2.369494e-08,1.049207e-06,7.617934e-07,52.020103,150.165798,47.989278,9.020000e-07,9.078333e-07,2
127096,2021-02-17,PIVX_BTC,0.000014,0.000017,0.000013,0.000016,2119250.0,-2.938339e-08,1.729947e-06,1.592053e-06,28.651403,-64.018692,6.460573,1.750000e-06,1.750000e-06,2
127097,2021-02-17,STORJ_BTC,0.000016,0.000016,0.000014,0.000015,3600209.0,2.369494e-08,1.049207e-06,7.617934e-07,52.020103,150.165798,47.989278,9.020000e-07,9.078333e-07,2
127098,2021-02-17,CELO_BTC,0.000093,0.000108,0.000085,0.000092,1715704.9,-1.143647e-09,2.754664e-07,2.085336e-07,42.078085,5.706760,2.660642,2.326667e-07,2.486364e-07,2
127099,2021-02-17,AGI_BTC,0.000004,0.000004,0.000003,0.000004,21452822.0,-2.590028e-08,2.703575e-07,1.526425e-07,37.855863,-82.605242,31.727292,2.356667e-07,2.771667e-07,2


In [ ]:
processed_full.isna().any()

date            False
tic             False
open            False
high            False
low             False
close           False
volume          False
macd            False
boll_ub         False
boll_lb         False
rsi_30          False
cci_30          False
dx_30           False
close_30_sma    False
close_60_sma    False
day             False
dtype: bool

In [ ]:
#### DATA SPLIT TRAIN TRADE RATIO 70/30

trade_len = int(len(processed_full)*0.7)
train_len = int(len(processed_full)*0.3+1)
total = trade_len+train_len

In [ ]:
train = data_split(processed_full, processed_full.date.min(),processed_full.date.loc[trade_len])
trade = data_split(processed_full, processed_full.date.loc[trade_len],processed_full.date.max())
print(len(train))
print(len(trade))
print(f'trade length:{trade_len}, train length: {train_len}, for total len of {total} of {len(processed_full)}')


88900
38100
trade length:88970, train length: 38131, for total len of 127101 of 127100


In [ ]:
train

,date,tic,open,high,low,close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,day
0,2017-08-27,ADA_BTC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,6
0,2017-08-27,AERGO_BTC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,6
0,2017-08-27,AGI_BTC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,6
0,2017-08-27,AKRO_BTC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,6
0,2017-08-27,AMB_BTC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,2020-02-01,XEM_BTC,4.820000e-06,5.430000e-06,4.790000e-06,5.280000e-06,58116266.0,-3.852414e-08,1.107707e-06,8.272934e-07,31.996931,-174.624645,34.743536,9.903333e-07,1.009697e-06,5
888,2020-02-01,XLM_BTC,6.480000e-06,6.690000e-06,6.460000e-06,6.630000e-06,38325484.0,-1.053946e-09,3.504154e-07,2.565846e-07,45.041296,127.192982,24.750103,2.990000e-07,3.798333e-07,5
888,2020-02-01,XVG_BTC,4.000000e-07,4.300000e-07,4.000000e-07,4.300000e-07,96854241.0,2.154301e-08,4.629778e-07,3.110222e-07,53.957303,52.554316,27.995513,3.863333e-07,3.368333e-07,5
888,2020-02-01,YOYOW_BTC,1.140000e-06,1.170000e-06,1.130000e-06,1.160000e-06,4106977.0,1.967765e-08,3.888191e-07,2.101809e-07,55.429784,220.535714,56.650906,2.943333e-07,3.028333e-07,5


In [ ]:
len(train.tic.unique())

100

In [ ]:
config["TECHNICAL_INDICATORS_LIST"]
len(config["TECHNICAL_INDICATORS_LIST"])

8

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config["TECHNICAL_INDICATORS_LIST"])*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config["TECHNICAL_INDICATORS_LIST"], 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

Stock Dimension: 100, State Space: 1001


In [ ]:
env_train, _ = e_train_gym.get_sb_env()
# print(type(env_train))
# print(dir(env_train))
# print(dir(env_train.observation_space))
# print(env_train.observation_space.low.shape)

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 1000,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)


{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 1000}
Using cpu device


In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=1000000)

Streaming output truncated to the last 5000 lines.
|    loss                 | -1.91       |
|    n_updates            | 3020        |
|    policy_gradient_loss | -0.0169     |
|    std                  | 1.58        |
|    value_loss           | 1.25e-05    |
-----------------------------------------
day: 888, episode: 700
begin_total_asset: 1000000.00
end_total_asset: 999991.80
total_reward: -8.20
total_cost: 0.16
total_trades: 38908
Sharpe: -1.303
-----------------------------------------
| environment/            |             |
|    portfolio_value      | 1e+06       |
|    total_cost           | 0.171       |
|    total_reward         | -12.9       |
|    total_reward_pct     | -0.00129    |
|    total_trades         | 39304       |
| time/                   |             |
|    fps                  | 74          |
|    iterations           | 304         |
|    time_elapsed         | 8408        |
|    total_timesteps      | 622592      |
| train/                  |             |

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model_ppo, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.head()

In [ ]:
df_actions.head()

In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')